In [7]:
# Import libraries
import pandas as pd
import json
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text
from datetime import date
import random
from datetime import datetime

In [8]:
df_answers = pd.read_csv("dataset/Answers.csv", encoding="ISO-8859-1").dropna()
df_questions = pd.read_csv("dataset/Questions.csv", encoding="ISO-8859-1").dropna()
df_taglink = pd.read_csv("dataset/Tags.csv", encoding="ISO-8859-1").dropna()
df_tags = df_taglink.drop_duplicates(subset=['Tag'], keep='first').drop(columns=['Id'])
df_tags.head()

,Tag
0,python
1,osx
2,fonts
3,photoshop
5,windows


In [9]:
print(f'df_questions')
print({df_questions.info()})
print("\n------------------------------------------\n")
print(f'df_answers')
print(f'{df_answers.info()}')
print("\n------------------------------------------\n")
print(f'df_tags')
print(f'{df_tags.info()}')
print("\n------------------------------------------\n")
print(f'df_taglink')
print(f'{df_taglink.info()}')

df_questions
<class 'pandas.core.frame.DataFrame'>
Index: 601070 entries, 0 to 607281
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            601070 non-null  int64  
 1   OwnerUserId   601070 non-null  float64
 2   CreationDate  601070 non-null  object 
 3   Score         601070 non-null  int64  
 4   Title         601070 non-null  object 
 5   Body          601070 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 32.1+ MB
{None}

------------------------------------------

df_answers
<class 'pandas.core.frame.DataFrame'>
Index: 981755 entries, 0 to 987121
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            981755 non-null  int64  
 1   OwnerUserId   981755 non-null  float64
 2   CreationDate  981755 non-null  object 
 3   ParentId      981755 non-null  int64  
 4   Score         981755 non-null 

In [10]:
df_questions['OwnerUserId'] = df_questions['OwnerUserId'].apply(int)
df_questions['Title'] = df_questions['Title'].astype('string')
df_questions['Body'] = df_questions['Body'].astype('string')
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate']).dt.strftime('%Y-%m-%d %H:%M:%S')

df_answers['OwnerUserId'] = df_answers['OwnerUserId'].apply(int)
df_answers['Body'] = df_answers['Body'].astype('string')
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
print("Tables before filtering:")
print(df_questions.shape)
print(df_answers.shape)
print(df_tags.shape)
print(df_taglink.shape)

df_questions = df_questions[df_questions['CreationDate'].str.contains(r'^2015')]
df_answers = df_answers[df_answers['CreationDate'].str.contains(r'^2015')]
# df_questions = df_questions[df_questions['CreationDate'].dt.year == 2015]
# df_answers= df_answers[df_answers['CreationDate'].dt.year == 2015]
df_taglink = df_taglink[df_taglink['Id'].isin(df_questions['Id'])]

print("\nTables after filtering:")
print(df_questions.shape)
print(df_answers.shape)
print(df_tags.shape)
print(df_taglink.shape)

Tables before filtering:
(601070, 6)
(981755, 6)
(16895, 1)
(1884635, 2)

Tables after filtering:
(132161, 6)
(193869, 6)
(16895, 1)
(414364, 2)


In [ ]:
#Connect to MySQL

# database_name = 'stackoverflow'
# Create Connection
# connection = mysql.connector.connect(
#   host="localhost",
#   user="root", 
#   password = '!Leandro02#'
# )

# cursor = connection.cursor()
# cursor.execute(f"DROP DATABASE IF EXISTS {database_name}")
# cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
# cursor.execute(f"USE {database_name}")


# cursor.execute("DROP TABLE IF EXISTS taglink")
# cursor.execute("DROP TABLE IF EXISTS tags")
# cursor.execute("DROP TABLE IF EXISTS answers")
# cursor.execute("DROP TABLE IF EXISTS questions")

#Creating Table
# cursor.execute("""CREATE TABLE questions (
#                Id INT AUTO_INCREMENT PRIMARY KEY, 
#                OwnerUserId INT, 
#                CreationDate DATETIME, 
#                Score INT, 
#                Title VARCHAR(255), 
#                Body TEXT)""")
# cursor.execute("""CREATE TABLE answers (
#                Id INT AUTO_INCREMENT PRIMARY KEY,
#                OwnerUserId INT, 
#                CreationDate DATETIME,
#                ParentId INT, 
#                Score INT, 
#                Body TEXT,
#                FOREIGN KEY (ParentId) REFERENCES questions(Id))""")
# cursor.execute("CREATE TABLE tags (Tag VARCHAR(255) PRIMARY KEY)")
# cursor.execute("""CREATE TABLE taglink (
#                Id INT, 
#                Tag VARCHAR(255),
#                FOREIGN KEY (Id) REFERENCES questions(Id),
#                FOREIGN KEY (Tag) REFERENCES tags(Tag))""")

# connection.commit()

#Check if Table Exists
# cursor.execute("SHOW TABLES")
# for x in cursor:
#     print(x)


# def insertData(insert_query, data, table_name, batch_size=1000):
#     for i in range(0, len(data), batch_size):
#         batch = data[i:i + batch_size]
#         cursor.executemany(insert_query, batch)
#         connection.commit()
#         print(f"Inserted {cursor.rowcount} rows into MySQL, in table {table_name}.")

# Insert data
# insert_questions = """
# INSERT INTO questions (Id, OwnerUserId, CreationDate, Score, Title, Body)
# VALUES (%s, %s, %s, %s, %s, %s);
# """
# data_to_insert = [(item["Id"], item["OwnerUserId"],item["CreationDate"], item["Score"], item["Title"], item["Body"]) for item in df_questions.to_dict(orient="records")]
# insertData(insert_questions, data_to_insert, "Questions")

# insert_answers = """
# INSERT INTO answers (Id, OwnerUserId, CreationDate, ParentId, Score, Body)
# VALUES (%s, %s, %s, %s, %s, %s);
# """
# data_to_insert = [(item["Id"], item["OwnerUserId"],item["CreationDate"], item["ParentId"], item["Score"], item["Body"]) for item in df_answers.to_dict(orient="records")]
# insertData(insert_answers, data_to_insert, "Answers")

# insert_tags = """
# INSERT INTO tags (Tag)
# VALUES (%s);
# """
# data_to_insert = [(item["Tag"]) for item in df_tags.to_dict(orient="records")]
# insertData(insert_tags, data_to_insert, "Tags")

# insert_taglink = """
# INSERT INTO taglink (Id, Tag)
# VALUES (%s, %s);
# """
# data_to_insert = [(item["Id"], item["Tag"]) for item in df_taglink.to_dict(orient="records")]
# insertData(insert_taglink, data_to_insert, "TagLink")

In [11]:
#Connect to MySQL with slqAlchemy
username = 'root'
password = '!Leandro02#'
host = 'localhost' 
port = '3306' 
database_name = 'stackoverflow'
questions_table = 'questions'
answers_table = 'answers'
tags_table = 'tags'
taglink_table = 'taglink'

# Create the connection URL for SQL Alchemy
db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}'
engine = create_engine(db_url)
with engine.connect() as connection:
    connection.execute(text(f"DROP DATABASE IF EXISTS {database_name}"))
    connection.execute(text(f"CREATE DATABASE {database_name}"))
db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(db_url)


In [12]:
df_questions.to_sql(questions_table, con=engine, if_exists='replace', chunksize= 1000, index=False)
df_answers.to_sql(answers_table, con=engine, if_exists='replace', chunksize= 1000, index=False)
df_tags.to_sql(tags_table, con=engine, if_exists='replace', chunksize= 1000, index=False)
df_taglink.to_sql(taglink_table, con=engine, if_exists='replace', chunksize= 1000, index=False)

1884635

In [15]:
df_questions['answers'] = df_questions['Id'].apply(lambda x: list(df_answers[df_answers['ParentId'] == x]['Id']))
df_questions.head()


,Id,OwnerUserId,CreationDate,Score,Title,Body,answers
0,469,147,2008-08-02 15:11:16,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,"[497, 518, 3040, 195170]"
1,502,147,2008-08-02 17:01:58,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,"[536, 7073, 7090]"
2,535,154,2008-08-02 18:43:54,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,"[538, 541, 660, 61746, 74452, 9120453]"
3,594,116,2008-08-03 01:15:08,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,"[595, 25213, 125140]"
4,683,199,2008-08-03 13:19:16,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,"[701, 735, 745, 750, 31126, 31188, 57833, 4905..."


In [16]:
df_questions['tags'] = df_questions['Id'].apply(lambda x: list(df_taglink[df_taglink['Id'] == x]['Tag']))
df_questions.head()

KeyboardInterrupt: 

In [ ]:
#Convert CreationDate from string to datetime
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])

In [ ]:
# Setting MongoDB database

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")

# Drop and create database
client.drop_database("stackoverflow")
db = client['stackoverflow']

# Drop and create collections
db['Questions'].drop()
db['Answers'].drop()
db['Tags'].drop()
db['TagLink'].drop()
collection_questions = db['Questions']
collection_answers = db['Answers']
collection_tags = db['Tags']
collection_taglink = db['TagLink']

# Function to insert data into MongoDB
def insert_into_mongo(df, collection):
    initial_count = collection.count_documents({})
    print(f"# documents: {initial_count}") 
    try:
        data_dict = df.to_dict(orient="records")
        collection.insert_many(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted.")
    except Exception as e:
        attempted_count = len(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted")
        print(f"{attempted_count - valid_count} documents failed to insert")

insert_into_mongo(df_questions, collection_questions)
insert_into_mongo(df_answers, collection_answers)
insert_into_mongo(df_taglink, collection_taglink)
insert_into_mongo(df_tags, collection_tags)

# documents: 0
46722 documents inserted.
# documents: 0
65576 documents inserted.
# documents: 0
146233 documents inserted.
# documents: 0
16895 documents inserted.


In [13]:
queries = [
    {"description": "Retrieve title of all questions that have a score greater than 300", "query": ""},
    
    {"description": "Retrieve all answers from 2015", "query": ""},
    
    {"description": "Delete tag python on tags array in first question from 2015 that has at least 2 tags", "query": ""},

    {"description": "Insert answer for the most recent question from 2015 about PDF that have more than 4 answers", "query": ""}

]

In [ ]:
from bson.objectid import ObjectId
mongo_times = []
for q in queries:
    start_time = time.time()
    
    if "score greater" in q["description"]:
        query = {'Score': {'$gt': 300}}
        result = collection_questions.find(query)
        count = collection_questions.count_documents(query)

    elif "answers from 2015" in q["description"]:
        query = {
            'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            }
        }
        result = collection_answers.find(query)
        count = collection_answers.count_documents(query)

    elif "2 tags" in q["description"]:
        query = {
            'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            },
            '$expr': {'$gte': [{'$size': '$tags'}, 2]}
        }
        result = collection_questions.find_one(query, sort=[('CreationDate', 1)])
        if result:
            collection_questions.update_one(
                {'_id': result['_id']},
                {'$pop': {'tags': -1}}
            )
            collection_taglink.delete_one({'Id': result['Id']})
            print(f"The deleted tag is: {result['tags'][-1]}, the question ID is: {result['_id']}")

        count = collection_questions.count_documents(query)
    else:
        query = {'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            }, 
            'Title': {'$regex' : '.*pdf.*'}, 
            '$expr': {'$gte': [{'$size': '$answers'}, 4]}
            }
        result = collection_questions.find_one(query, sort=[('CreationDate', 1)])
        if result:
            last_id = collection_answers.find_one(sort=[('Id', -1)])['Id']
            awnser_id = ObjectId()
            parent_id = result['Id']
            parent_object_id = result['_id']
            new_answer = {
                '_id': awnser_id,
                'AnswerText': 'This is the new answer for this question.', 
                'CreationDate': datetime.now(),
                'OwnerUserId': random.randint(1, 1000),
                'ParentId': parent_id,
                'Score': 0,
                'Id': last_id + 1,
            }
            answer_result = collection_answers.insert_one(new_answer)
            answer_id = answer_result.inserted_id
            collection_questions.update_one({'_id': parent_object_id}, {'$push': {'answers': last_id+1}})
            print(f"Inserted new answer for question ID: {parent_object_id}, Answer ID: {answer_id}")
        count = collection_questions.count_documents(query)
    end_time = time.time()
    mongo_times.append(end_time - start_time)
    print(f"{q['description']}: {count} found in {end_time - start_time:.4f} seconds")

Retrieve all questions that have a score greater than 20: 57 found in 0.0265 seconds
Retrieve all answers from October of 2015: 17178 found in 0.0300 seconds
The deleted tag is: regex, the question ID is: 6749e74c5d62097d450e3f05
Delete first tag on tags array in first question from 2015 that has more than 2 tags: 43276 found in 0.1410 seconds
Inserted new answer for question ID: 6749e74c5d62097d450ec048, Answer ID: 6749e85f5d62097d451272e7
Insert answer for the most recent question from 2015 about PDF that have more than 4 answers: 2 found in 0.1940 seconds


In [ ]:
mysql_times = []
with engine.connect() as connection:
    for q in queries:
        start_time = time.time()
        if "score greater" in q["description"]:
            result = connection.execute(text("SELECT title FROM questions WHERE score > 300"))
            results =  result.fetchall()

        elif "answers from 2015" in q["description"]:
            result = connection.execute(text("SELECT * FROM answers WHERE creationdate LIKE '2015%'"))
            results =  result.fetchall()
        
        elif "2 tags" in q["description"]:
            result = connection.execute(text("""
                WITH qt AS (
                SELECT q.Id 
                FROM questions q
                JOIN taglink t ON q.Id = t.Id
                WHERE q.CreationDate LIKE '2015%'
                AND t.Tag = 'python'
                GROUP BY q.Id
                HAVING COUNT(t.Tag) >= 2
                ORDER BY MIN(q.CreationDate) DESC
                LIMIT 1)

                DELETE FROM taglink 
                WHERE id = (SELECT Id FROM qt) AND tag = 'python';
            """))
            results = []  # No rows to fetch for DELETE operation
        else:
            result = connection.execute(text("""
                INSERT INTO answers (OwnerUserId, CreationDate, ParentId, Score, Body)
                SELECT 1, NOW(), q.Id, 0, 'This is an answer'
                FROM questions q
                WHERE q.CreationDate LIKE '2015%'
                AND q.Title LIKE '%PDF%'
                AND (SELECT COUNT(*) FROM answers a WHERE a.ParentId = q.Id) > 4
                ORDER BY q.CreationDate DESC
                LIMIT 1
            """))
            results = []  # No rows to fetch for INSERT operation
        end_time = time.time()
        mysql_times.append(end_time - start_time)
        print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")

Retrieve title of all questions that have a score greater than 300: 241 results found in 2.8197 seconds
Retrieve all answers from 2015: 193869 results found in 3.3233 seconds
Delete tag python on tags array in first question from 2015 that has at least 2 tags: 0 results found in 7.7180 seconds
Insert answer for the most recent question from 2015 about PDF that have more than 4 answers: 0 results found in 1019.8602 seconds


In [ ]:
comparison_df = pd.DataFrame({
    "Query": [q["description"] for q in queries],
    "MongoDB_Time(s)": mongo_times,
    "MySQL_Time(s)": mysql_times
})

print(comparison_df)